In [1]:
import gym
import tensorflow as tf
import numpy as np
import copy
import random
from collections import deque
from gym.envs.toy_text.frozen_lake import FrozenLakeEnv
env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
#env =FrozenLakeEnv(is_slippery=False,map_name="4x4")
env.render()


SFFF
FHFH
FFFH
HFFG


In [2]:
lr=0.005


In [3]:
inputdim=env.observation_space.n
outputdim=env.action_space.n
hiddendim=100
print(inputdim, outputdim)
X=tf.placeholder(tf.float32, shape = [None,inputdim],name="X")
Y=tf.placeholder(tf.float32, shape = [None,outputdim],name="Y")
# Y=tf.placeholder(tf.float32,shape=[None,1],name="Y")
# action=tf.placeholder(tf.int32,shape=[1],name="action")

weight1=tf.get_variable("w1", initializer=tf.keras.initializers.he_uniform(), shape=[inputdim,hiddendim], dtype=tf.float32)
bias1=tf.get_variable("b1", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight2=tf.get_variable("w2", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias2=tf.get_variable("b2", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight3=tf.get_variable("w3", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias3=tf.get_variable("b3", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight4=tf.get_variable("w4", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,outputdim], dtype=tf.float32)
bias4=tf.get_variable("b4", initializer=tf.initializers.random_normal(0,0.00), shape=[outputdim], dtype=tf.float32)

#varlist=[weight1,weight2,weight3,weight4,bias1,bias2,bias3,bias4]
varlist=[weight1,weight4,bias1,bias4]
H1=tf.nn.relu(tf.nn.xw_plus_b(X,weight1,bias1))
#H2=tf.nn.relu(tf.nn.xw_plus_b(H1,weight2,bias2))
#H3=tf.nn.relu(tf.nn.xw_plus_b(H2,weight3,bias3))
#Y_pred=tf.nn.xw_plus_b(H3,weight4,bias4)
Y_pred=tf.nn.xw_plus_b(H1,weight4,bias4)


# Y=tf.placeholder(tf.float32,shape=[None,1],name="Y")
# action=tf.placeholder(tf.int32,shape=[1],name="action")

weight01=tf.get_variable("w01", initializer=tf.keras.initializers.he_uniform(), shape=[inputdim,hiddendim], dtype=tf.float32)
bias01=tf.get_variable("b01", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight02=tf.get_variable("w02", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias02=tf.get_variable("b02", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight03=tf.get_variable("w03", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,hiddendim], dtype=tf.float32)
bias03=tf.get_variable("b03", initializer=tf.initializers.random_normal(0,0.00), shape=[hiddendim], dtype=tf.float32)
weight04=tf.get_variable("w04", initializer=tf.keras.initializers.he_uniform(), shape=[hiddendim,outputdim], dtype=tf.float32)
bias04=tf.get_variable("b04", initializer=tf.initializers.random_normal(0,0.00), shape=[outputdim], dtype=tf.float32)

#varlist0=[weight01,weight02,weight03,weight04,bias01,bias02,bias03,bias04]
#varlist=[weight01,weight04,bias01,bias04]
H01=tf.nn.relu(tf.nn.xw_plus_b(X,weight01,bias01))
#H02=tf.nn.relu(tf.nn.xw_plus_b(H01,weight02,bias02))
#H03=tf.nn.relu(tf.nn.xw_plus_b(H02,weight03,bias03))
#Y_pred0=tf.nn.xw_plus_b(H03,weight04,bias04)
Y_pred0=tf.nn.xw_plus_b(H01,weight04,bias04)

assw1=weight01.assign(weight1)
assw2=weight02.assign(weight2)
assw3=weight03.assign(weight3)
assw4=weight04.assign(weight4)
assb1=bias01.assign(bias1)
assb2=bias02.assign(bias2)
assb3=bias03.assign(bias3)
assb4=bias04.assign(bias4)

#loss=tf.square(Y-Y_pred)
loss=tf.reduce_mean(tf.square(Y-Y_pred))
# loss=tf.square(Y-Y_pred[action])
#opt=tf.train.AdadeltaOptimizer(1.0).minimize(loss,var_list=varlist)
#opt=tf.train.GradientDescentOptimizer(lr).minimize(loss,var_list=varlist)
opt=tf.train.AdamOptimizer(lr).minimize(loss,var_list=varlist)
#opt=tf.train.AdamOptimizer(lr).minimize(loss)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)



16 4


In [44]:
max_episodes=300
max_step=100
gamma=0.995
epsilon0 = 1.0
sess.run(init)
sess.run([assw1,assw2,assw3,assw4,assb1,assb2,assb3,assb4])
# initialize

epsilon=epsilon0
min_step=max_step
min_step_episode=max_episodes
min_step_path=[]
max_replay=500
replay=deque(maxlen=max_replay)
#replay=[]
replay0=deque(maxlen=max_replay)
replay1=deque(maxlen=max_replay)
replay2=deque(maxlen=max_replay)
i_step=0
batch_size=10

for episode in range(max_episodes):
    epsilon=epsilon*0.99
    state = env.reset()
    total_reward = 0
    state_list = [state]
    win=0
    
    state_m=np.zeros([1,env.observation_space.n],dtype=np.float32)
    state_m[0][state]=1.0
    Q_pred=sess.run(Y_pred0,feed_dict={X:state_m})
    if np.random.rand()>epsilon :
        action=np.argmax(Q_pred[0])
    else:
        action=np.random.choice(env.action_space.n) 
    for step in range(1,max_step+1):
        
        i_step+=1
   
        state_next,reward,game_over,_ = env.step(action)
        
        if reward==0 : 
            reward=-0.01
        if reward!=1.0 and game_over==1:
            reward=-2
        
        state_m_next=np.zeros([1,env.observation_space.n],dtype=np.float32)
        state_m_next[0][state_next]=1.0
        Q_pred_next=sess.run(Y_pred0, feed_dict={X:state_m_next}) 
        if np.random.rand()>epsilon :
            action_next=np.argmax(Q_pred_next[0])
        else:
            action_next=np.random.choice(env.action_space.n)
        
        Y_true=copy.deepcopy(Q_pred[0])
        if reward==1.0 and game_over==1:
            Y_true[action]= reward
        else :
            Y_true[action]=reward+gamma* (Q_pred_next[0][action_next])
        Y_true=np.reshape(Y_true,[1,env.action_space.n])

        replay.append([state_m[0],Y_true])

#        if state in replay0:
#            index=replay0.index(state)
#            replay[index]=[state_m[0],Y_true]
#        else:
#            replay0.append(state)
#            replay.append([state_m[0],Y_true]) 
        
#            game_over=0
        # Q-Learning
        if i_step%20==0 and i_step!=0 and len(replay)>batch_size:
#            print(replay0)
            mini_batch = random.sample(replay,batch_size)
            states=np.zeros([batch_size,inputdim])
            Qs=np.zeros([batch_size,outputdim])
            for m in range(batch_size):
                states[m]=mini_batch[m][0]
                Qs[m]=mini_batch[m][1]
            cost0= sess.run([loss],feed_dict={X:states,Y:Qs})
            for i in range(0,2):
                sess.run([opt],feed_dict={X:states,Y:Qs})
                cost= sess.run([loss],feed_dict={X:states,Y:Qs})
#            Q_pred=sess.run(Y_pred,feed_dict={X:states})
            #print(i_step,cost0,cost,Qs[0],Q_pred[0])
#            print(i_step,cost0,cost)

        if i_step%100==0  :
#            print(i_step,"update")
            sess.run([assw1,assw2,assw3,assw4,assb1,assb2,assb3,assb4])
            
        state=state_next
        state_m=state_m_next
        action=action_next
        
        total_reward=total_reward+reward
        state_list.append(state)
        if game_over and reward==1:
            win=1
            if step< min_step:
                min_step=step
                min_step_path=state_list
                min_step_episode=episode
        if game_over==1:
            break
            

    line_out="Episode: %d, step: %d, total_reward: %f, epsilon: %f" %(episode, step, total_reward,epsilon)
    print(line_out)
    if reward==1 :
        print(state_list)
print( "min step: ", min_step, "min step episode", min_step_episode)
print("shortest path: ", min_step_path)
#print(Q)

Episode: 0, step: 3, total_reward: -2.020000, epsilon: 0.990000
Episode: 1, step: 2, total_reward: -2.010000, epsilon: 0.980100
Episode: 2, step: 14, total_reward: -2.130000, epsilon: 0.970299
Episode: 3, step: 14, total_reward: 0.870000, epsilon: 0.960596
[0, 0, 0, 0, 4, 8, 9, 13, 13, 13, 14, 14, 14, 14, 15]
Episode: 4, step: 2, total_reward: -2.010000, epsilon: 0.950990
Episode: 5, step: 2, total_reward: -2.010000, epsilon: 0.941480
Episode: 6, step: 10, total_reward: -2.090000, epsilon: 0.932065
Episode: 7, step: 11, total_reward: -2.100000, epsilon: 0.922745
Episode: 8, step: 5, total_reward: -2.040000, epsilon: 0.913517
Episode: 9, step: 21, total_reward: 0.800000, epsilon: 0.904382
[0, 0, 0, 0, 4, 0, 0, 0, 4, 0, 1, 2, 2, 3, 2, 6, 10, 14, 14, 13, 14, 15]
Episode: 10, step: 2, total_reward: -2.010000, epsilon: 0.895338
Episode: 11, step: 5, total_reward: -2.040000, epsilon: 0.886385
Episode: 12, step: 2, total_reward: -2.010000, epsilon: 0.877521
Episode: 13, step: 4, total_reward:

Episode: 131, step: 3, total_reward: -2.020000, epsilon: 0.265366
Episode: 132, step: 3, total_reward: -2.020000, epsilon: 0.262713
Episode: 133, step: 4, total_reward: -2.030000, epsilon: 0.260085
Episode: 134, step: 3, total_reward: -2.020000, epsilon: 0.257485
Episode: 135, step: 3, total_reward: -2.020000, epsilon: 0.254910
Episode: 136, step: 4, total_reward: -2.030000, epsilon: 0.252361
Episode: 137, step: 3, total_reward: -2.020000, epsilon: 0.249837
Episode: 138, step: 5, total_reward: -2.040000, epsilon: 0.247339
Episode: 139, step: 5, total_reward: -2.040000, epsilon: 0.244865
Episode: 140, step: 3, total_reward: -2.020000, epsilon: 0.242417
Episode: 141, step: 3, total_reward: -2.020000, epsilon: 0.239992
Episode: 142, step: 4, total_reward: -2.030000, epsilon: 0.237593
Episode: 143, step: 3, total_reward: -2.020000, epsilon: 0.235217
Episode: 144, step: 2, total_reward: -2.010000, epsilon: 0.232864
Episode: 145, step: 7, total_reward: -2.060000, epsilon: 0.230536
Episode: 1

Episode: 241, step: 3, total_reward: -2.020000, epsilon: 0.087845
Episode: 242, step: 3, total_reward: -2.020000, epsilon: 0.086967
Episode: 243, step: 3, total_reward: -2.020000, epsilon: 0.086097
Episode: 244, step: 3, total_reward: -2.020000, epsilon: 0.085236
Episode: 245, step: 3, total_reward: -2.020000, epsilon: 0.084384
Episode: 246, step: 3, total_reward: -2.020000, epsilon: 0.083540
Episode: 247, step: 3, total_reward: -2.020000, epsilon: 0.082704
Episode: 248, step: 3, total_reward: -2.020000, epsilon: 0.081877
Episode: 249, step: 3, total_reward: -2.020000, epsilon: 0.081059
Episode: 250, step: 6, total_reward: 0.950000, epsilon: 0.080248
[0, 4, 8, 9, 13, 14, 15]
Episode: 251, step: 3, total_reward: -2.020000, epsilon: 0.079445
Episode: 252, step: 3, total_reward: -2.020000, epsilon: 0.078651
Episode: 253, step: 3, total_reward: -2.020000, epsilon: 0.077864
Episode: 254, step: 4, total_reward: -2.030000, epsilon: 0.077086
Episode: 255, step: 3, total_reward: -2.020000, epsi

In [25]:
#state=np.arange(0,15)
#sess.run(init)
state_m=np.zeros([16,env.observation_space.n],dtype=np.float32)
for i in range(0,15):
    state_m[i][i]=1.0
print(state_m)
Q_pred=sess.run(Y_pred,feed_dict={X:state_m})
print(Q_pred)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[-0.3193047   0.0772663   0.12334345  0.1268539 ]
 [-0.32311273  0.09331876  0.17124411  0.11340132]
 [-0.3256859   0.06620821  0.1321699   0.12647435]
 [-0.32298675  0.05508754  0.1